# Creación de la Columna Cuentas_Diarias - Telecom X

## Propósito
En este cuaderno, se crea la columna `Cuentas_Diarias` para calcular el valor diario de facturación de los clientes, dividiendo `account_Charges.Monthly` por 30. Esto proporciona una visión detallada del comportamiento de facturación de los clientes a lo largo del tiempo.

## Tarea
1. Cargar el dataset limpio desde la URL de la API y aplicar correcciones previas.
2. Crear la columna `Cuentas_Diarias` dividiendo `account_Charges.Monthly` por 30.
3. Verificar la coherencia de la nueva columna (tipo de datos, valores no negativos).
4. Guardar el dataset actualizado con la nueva columna.

In [1]:
# Importación de bibliotecas
import pandas as pd
import requests
import json

# Carga de datos desde la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()

# Conversión a DataFrame
df = pd.DataFrame(data)

# Desanidar columnas JSON
customer_df = pd.json_normalize(df['customer'])
customer_df.columns = [f"customer_{col}" for col in customer_df.columns]
phone_df = pd.json_normalize(df['phone'])
phone_df.columns = [f"phone_{col}" for col in phone_df.columns]
internet_df = pd.json_normalize(df['internet'])
internet_df.columns = [f"internet_{col}" for col in internet_df.columns]
account_df = pd.json_normalize(df['account'])
account_df.columns = [f"account_{col}" for col in account_df.columns]

# Combinar los datos desanidados
df_clean = pd.concat([df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

# Aplicar correcciones previas
df_clean['Churn'] = df_clean['Churn'].replace('', 'No')
df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')
df_clean['customer_tenure'] = df_clean['customer_tenure'].astype('int64')
df_clean['customer_SeniorCitizen'] = df_clean['customer_SeniorCitizen'].astype('int64')

# Imputar valores nulos en 'account_Charges.Total'
median_total = df_clean['account_Charges.Total'].median()
df_clean['account_Charges.Total'] = df_clean.apply(
    lambda row: row['account_Charges.Monthly'] if pd.isna(row['account_Charges.Total']) and row['customer_tenure'] <= 1
    else row['account_Charges.Total'] if not pd.isna(row['account_Charges.Total'])
    else median_total,
    axis=1
)

# 1. Crear la columna 'Cuentas_Diarias'
df_clean['Cuentas_Diarias'] = df_clean['account_Charges.Monthly'] / 30

# 2. Verificar la nueva columna
print("Estadísticas descriptivas de 'Cuentas_Diarias':")
print(df_clean['Cuentas_Diarias'].describe())

# Verificar valores nulos en 'Cuentas_Diarias'
print("\nValores nulos en 'Cuentas_Diarias':")
print(df_clean['Cuentas_Diarias'].isnull().sum())

# Verificar valores negativos en 'Cuentas_Diarias'
if (df_clean['Cuentas_Diarias'] < 0).any():
    print("\nValores negativos encontrados en 'Cuentas_Diarias':")
    print(df_clean[df_clean['Cuentas_Diarias'] < 0]['Cuentas_Diarias'])
else:
    print("\nNo se encontraron valores negativos en 'Cuentas_Diarias'.")

# 3. Verificación de tipos de datos
print("\nTipos de datos del DataFrame actualizado:")
print(df_clean.dtypes)

# 4. Mostrar las primeras filas con la nueva columna
print("\nPrimeras 5 filas del DataFrame con 'Cuentas_Diarias':")
print(df_clean[['customerID', 'account_Charges.Monthly', 'Cuentas_Diarias']].head())

# 5. Guardar el dataset actualizado
df_clean.to_csv('TelecomX_Data_with_daily.csv', index=False)
print("\nDataset actualizado guardado como 'TelecomX_Data_with_daily.csv'.")

Estadísticas descriptivas de 'Cuentas_Diarias':
count    7267.000000
mean        2.157337
std         1.004319
min         0.608333
25%         1.180833
50%         2.343333
75%         2.995833
max         3.958333
Name: Cuentas_Diarias, dtype: float64

Valores nulos en 'Cuentas_Diarias':
0

No se encontraron valores negativos en 'Cuentas_Diarias'.

Tipos de datos del DataFrame actualizado:
customerID                    object
Churn                         object
customer_gender               object
customer_SeniorCitizen         int64
customer_Partner              object
customer_Dependents           object
customer_tenure                int64
phone_PhoneService            object
phone_MultipleLines           object
internet_InternetService      object
internet_OnlineSecurity       object
internet_OnlineBackup         object
internet_DeviceProtection     object
internet_TechSupport          object
internet_StreamingTV          object
internet_StreamingMovies      object
account_Contr

## Descripción de la tarea
En esta sección, se creó la columna `Cuentas_Diarias` dividiendo `account_Charges.Monthly` por 30 para obtener el valor diario de facturación. Se verificó que la nueva columna no tenga valores nulos ni negativos, y que su tipo de datos sea correcto (`float64`). El dataset actualizado se guardó como `TelecomX_Data_with_daily.csv` para su uso en el análisis exploratorio.